In [1]:
import json

In [2]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
print(type(documents))
print('------')
print(len(documents))
print('------')
print(documents[0])
print('------')
print(documents[0]['text'])
print('------')

<class 'list'>
------
948
------
{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}
------
The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this

### Comandos docker para correr una instancia de elasticsearch en docker

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    -v /media/villarreal-fx/carlos-011/data_elasticsearch:/usr/share/elasticsearch/data \  # Creando volumen para partición montada en media
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

### Verificar el estado del cluster
```bash
curl -XGET 'http://localhost:9200/_cluster/health?pretty'
```

```bash
{
  "cluster_name" : "docker-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 0,
  "active_shards" : 0,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "unassigned_primary_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}
```

### 1. Verifica si el índice existe y su estructura:

```bash
curl -XGET 'http://localhost:9200/_cat/indices?v'
```

### 2. Consulta el mapeo del índice:

```bash
curl -XGET 'http://localhost:9200/course-questions/_mapping?pretty'
```
### 3. Lista todos los documentos del índice:

```bash
curl -XGET 'http://localhost:9200/course-questions/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": { "match_all": {} },
  "size": 10
}'
```
### 4. Consulta específica para depurar

```bash
curl -XGET 'http://localhost:9200/course-questions/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool": {
      "must": { "match_all": {} },
      "filter": { "term": { "course": "data-engineering-zoomcamp" } }
    }
  }
}'
```





## Creación de indices

In [22]:

from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200', request_timeout=60) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.ignore_malformed": False,  # Evita mapeo automático
        "index.mapping.coerce": False  # Evita conversiones automáticas
    },
    "mappings": {
        "dynamic": "strict",  # Bloquea campos no definidos
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

try:
    # Intenta borrar el índice (ignora si no existe)
    print(f"Intentando borrar el índice '{index_name}' si existe...")
    es_client.indices.delete(index=index_name, ignore_unavailable=True)
    print(f"Índice '{index_name}' borrado o no existía.")

    # Intenta crear el índice
    print(f"Intentando crear el índice '{index_name}'...")
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"¡Índice '{index_name}' creado exitosamente!")
except Exception as e:
    print(f"ERROR: Ocurrió un problema al interactuar con Elasticsearch: {e}")

Intentando borrar el índice 'course-questions' si existe...
Índice 'course-questions' borrado o no existía.
Intentando crear el índice 'course-questions'...
¡Índice 'course-questions' creado exitosamente!


In [23]:
!curl -XGET 'http://localhost:9200/course-questions/_mapping?pretty'

{
  "course-questions" : {
    "mappings" : {
      "dynamic" : "strict",
      "properties" : {
        "course" : {
          "type" : "keyword"
        },
        "id" : {
          "type" : "keyword"
        },
        "question" : {
          "type" : "text"
        },
        "section" : {
          "type" : "text"
        },
        "text" : {
          "type" : "text"
        }
      }
    }
  }
}


In [24]:

from tqdm.auto import tqdm

# Función para indexar documentos con manejo de errores
def index_documents(documents):
    successes = 0
    errors = 0
    
    for doc in tqdm(documents, desc="Indexando documentos"):
        try:
            # Asegúrate que el documento tenga todos los campos requeridos
            required_fields = ['text', 'section', 'question', 'course', 'id']
            if not all(field in doc for field in required_fields):
                print(f"Documento falta campos requeridos: {doc}")
                errors += 1
                continue
                
            # Intenta indexar el documento
            response = es_client.index(
                index=index_name,
                document=doc,
                id=doc['id']  # Usamos el campo 'id' como ID del documento
            )
            
            if response['result'] in ['created', 'updated']:
                successes += 1
            else:
                errors += 1
                print(f"Respuesta inesperada: {response}")
                
        except Exception as e:
            errors += 1
            print(f"Error indexando documento {doc.get('id', 'sin ID')}: {str(e)}")
    
    print(f"\nResumen: {successes} documentos indexados, {errors} errores")

In [25]:
!curl -XGET 'http://localhost:9200/_cluster/health?pretty'

{
  "cluster_name" : "docker-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 1,
  "active_shards" : 1,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "unassigned_primary_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


In [26]:
index_documents(documents)

Indexando documentos:   0%|          | 0/948 [00:00<?, ?it/s]


Resumen: 948 documentos indexados, 0 errores


In [27]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    try:
        response = es_client.search(index=index_name, body=search_query)
        print(len(response))
    except Exception as e:
        print(f"Error al realizar la búsqueda: {str(e)}")
        return []
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [28]:
elastic_search(
    query="Course - I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

4


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

In [29]:
import pandas as pd

In [30]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [32]:
df_ground_truth.head()

,question,course,document
0,What is the exact start date and time for the ...,data-engineering-zoomcamp,c02e79ef
1,How can I stay updated with the course schedul...,data-engineering-zoomcamp,c02e79ef
2,Where do I need to register before the course ...,data-engineering-zoomcamp,c02e79ef
3,Is there a specific platform or channel for co...,data-engineering-zoomcamp,c02e79ef
4,What should I do to ensure I don't miss the be...,data-engineering-zoomcamp,c02e79ef


In [38]:
df_ground_truth.shape

(10, 3)

In [31]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [33]:
ground_truth

[{'question': 'What is the exact start date and time for the course?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'How can I stay updated with the course schedule and important announcements?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'Where do I need to register before the course begins?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'Is there a specific platform or channel for course-related communication?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': "What should I do to ensure I don't miss the beginning of the course?",
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'What background knowledge or skills do I need before enrolling in this course?',
  'course': 'data-engineering-zoomcamp',
  'document': '1f6520ca'},
 {'question': 'Are there any specific technical requirements I should meet to take 

In [37]:
len(ground_truth)

10

In [36]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/10 [00:00<?, ?it/s]

4
4
4
4
4
4
4
4
4
4


In [39]:
relevance_total

[[False, False, True, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, True, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False]]

In [40]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [41]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [42]:
hit_rate(relevance_total), mrr(relevance_total)

(0.4, 0.26666666666666666)